# Демо: Решение обратной задачи ЭЭГ с физически информированными нейронными сетями

Этот ноутбук демонстрирует полный процесс решения обратной задачи электроэнцефалографии (ЭЭГ) с использованием:
- Синтетической генерации данных ЭЭГ
- Физически информированных нейронных сетей (PINNs)
- Обратного решения уравнения Пуассона для восстановления потенциала мозга

## Обратная задача ЭЭГ

В прямой задаче мы знаем источники в мозге и вычисляем потенциалы на сенсорах:
$$ \nabla^2 \phi = -\rho $$

В обратной задаче мы знаем измерения на сенсорах и должны восстановить $\phi(x,y,z,t)$ во всем объеме.

In [1]:
# Импорты и настройка
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import json
import os
from pathlib import Path
import seaborn as sns

# Настройка стиля
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("📊 Импорты загружены")

📊 Импорты загружены


In [3]:
# Создание синтетических данных ЭЭГ
from inverse_npde.eeg_data_generator import generate_eeg_data, create_sensor_grid

# Параметры генерации
n_sensors = 64  # Количество электродов ЭЭГ
n_time_steps = 50  # Временные точки
spatial_domain = (-1.0, 1.0, -1.0, 1.0)  # Пространственная область (x_min, x_max, y_min, y_max)
temporal_domain = (0.0, 1.0)  # Временная область

print(f"🔬 Генерация синтетических данных ЭЭГ...")
print(f"   • {n_sensors} сенсоров")
print(f"   • {n_time_steps} временных шагов")
print(f"   • Пространственная область: {spatial_domain}")
print(f"   • Временная область: {temporal_domain}")

# Генерация данных
eeg_data = generate_eeg_data(
    n_sensors=n_sensors,
    n_time_steps=n_time_steps,
    spatial_domain=spatial_domain,
    temporal_domain=temporal_domain
)

print("✅ Синтетические данные ЭЭГ созданы")

ImportError: cannot import name 'generate_eeg_data' from 'inverse_npde.eeg_data_generator' (/home/sasha/inverse-npde/src/inverse_npde/eeg_data_generator.py)

In [ ]:
# Визуализация синтетических данных ЭЭГ
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Расположение сенсоров
ax1 = axes[0, 0]
scatter = ax1.scatter(eeg_data['sensor_positions'][:, 0], eeg_data['sensor_positions'][:, 1], 
                     c=eeg_data['measured_potentials'][:, 0], cmap='RdBu_r', s=50)
ax1.set_title('Расположение сенсоров ЭЭГ\n(цвет = потенциал при t=0)', fontsize=12)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax1, label='Потенциал (мВ)')

# 2. Временные ряды для нескольких сенсоров
ax2 = axes[0, 1]
n_sensors_to_plot = 8
sensor_indices = np.linspace(0, n_sensors-1, n_sensors_to_plot, dtype=int)
for i, sensor_idx in enumerate(sensor_indices):
    ax2.plot(eeg_data['time_points'], eeg_data['measured_potentials'][sensor_idx, :], 
             label=f'Сенсор {sensor_idx+1}', alpha=0.7, linewidth=1.5)
ax2.set_title(f'Временные ряды потенциала\n({n_sensors_to_plot} случайных сенсоров)', fontsize=12)
ax2.set_xlabel('Время (с)')
ax2.set_ylabel('Потенциал (мВ)')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.grid(True, alpha=0.3)

# 3. Спектрограмма для центрального сенсора
ax3 = axes[1, 0]
central_sensor = n_sensors // 2
from scipy import signal
f, t_spec, Sxx = signal.spectrogram(eeg_data['measured_potentials'][central_sensor, :], 
                                   fs=n_time_steps/temporal_domain[1])
im = ax3.pcolormesh(t_spec, f[:20], np.log10(Sxx[:20, :]), shading='gouraud', cmap='viridis')
ax3.set_title(f'Спектрограмма центрального сенсора (#{central_sensor+1})', fontsize=12)
ax3.set_xlabel('Время (с)')
ax3.set_ylabel('Частота (Гц)')
plt.colorbar(im, ax=ax3, label='Логарифм мощности')

# 4. Статистика по сенсорам
ax4 = axes[1, 1]
sensor_means = np.mean(eeg_data['measured_potentials'], axis=1)
sensor_stds = np.std(eeg_data['measured_potentials'], axis=1)
ax4.scatter(sensor_means, sensor_stds, alpha=0.6, s=50)
ax4.set_title('Статистика по сенсорам', fontsize=12)
ax4.set_xlabel('Средний потенциал (мВ)')
ax4.set_ylabel('Стандартное отклонение (мВ)')
ax4.grid(True, alpha=0.3)

# Добавление аннотаций
ax4.annotate(f'Центральный\nсенсор #{central_sensor+1}', 
             xy=(sensor_means[central_sensor], sensor_stds[central_sensor]),
             xytext=(10, 10), textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5),
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

plt.tight_layout()
plt.savefig('figures/eeg_data_overview.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Визуализация синтетических данных ЭЭГ создана")

In [ ]:
# Запуск обратной задачи ЭЭГ
from src.inverse_npde.julia_interface import run_eeg_inverse_problem

print("🚀 Запуск эксперимента обратной задачи ЭЭГ...")
print("⏳ Это может занять несколько минут...")

# Настройки нейронной сети
nn_config = {
    "chain": [4, 32, 32, 8],  # [входные_размеры, скрытые_слои, выходные_размеры]
    "activation": "tanh",
    "initializer": "Glorot",
    "device": "cpu"  # Можно изменить на "cuda" если доступно
}

# Настройки оптимизации
opt_config = {
    "optimizer": "ADAM",
    "learning_rate": 0.001,
    "max_iterations": 1000,
    "batch_size": 100,
    "logging": True
}

# Настройки функции потерь
loss_config = {
    "physics_weight": 1.0,
    "boundary_weight": 1.0,
    "data_weight": 10.0,  # Высокий вес для данных, так как у нас есть измерения
    "initial_weight": 1.0
}

# Настройки домена
domain_config = {
    "space_bounds": [[-1.0, 1.0], [-1.0, 1.0]],  # x, y границы
    "time_bounds": [0.0, 1.0],
    "n_points": 1000,  # Количество коллокационных точек
    "sampling_strategy": "quasi-random"
}

try:
    # Запуск обратной задачи
    results = run_eeg_inverse_problem(
        nn_config=nn_config,
        opt_config=opt_config, 
        loss_config=loss_config,
        domain_config=domain_config,
        measured_potentials=eeg_data['measured_potentials'],
        sensor_positions=eeg_data['sensor_positions'],
        time_points=eeg_data['time_points']
    )
    
    print("✅ Эксперимент завершен успешно!")
    
except Exception as e:
    print(f"❌ Ошибка при решении: {e}")
    print("💡 Попробуйте уменьшить количество итераций или изменить параметры")
    raise

In [ ]:
# Анализ и визуализация результатов
print("📊 Анализ результатов обратной задачи...")

# Проверяем структуру результатов
print("Структура результатов:")
for key, value in results.items():
    if isinstance(value, dict):
        print(f"  {key}: словарь с ключами {list(value.keys())}")
    elif isinstance(value, np.ndarray):
        print(f"  {key}: массив {value.shape}")
    else:
        print(f"  {key}: {type(value).__name__}")

In [ ]:
# 1. Временная эволюция потенциала для одного сенсора
print("📈 Создание графика временной эволюции потенциала...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Выбираем центральный сенсор для анализа
central_sensor_idx = n_sensors // 2

# Извлекаем данные для центрального сенсора
sensor_positions = results['sensor_data']['positions']
true_potential_temporal = results['sensor_data']['true_potential_temporal']
predicted_potential_temporal = results['sensor_data']['predicted_potential_temporal']
time_points = results['sensor_data']['time_points']
mse_history = results['sensor_data']['mse_history']

# 1. Временная эволюция для центрального сенсора
ax1 = axes[0, 0]
ax1.plot(time_points, true_potential_temporal[central_sensor_idx, :], 
         'b-', linewidth=2, label='Истинный потенциал', alpha=0.8)
ax1.plot(time_points, predicted_potential_temporal[central_sensor_idx, :], 
         'r--', linewidth=2, label='Восстановленный потенциал', alpha=0.8)
ax1.fill_between(time_points, 
                true_potential_temporal[central_sensor_idx, :],
                predicted_potential_temporal[central_sensor_idx, :],
                alpha=0.2, color='gray', label='Ошибка')
ax1.set_title(f'Временная эволюция потенциала\nСенсор #{central_sensor_idx+1} (центральный)', fontsize=12)
ax1.set_xlabel('Время (с)')
ax1.set_ylabel('Потенциал (мВ)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Ошибка MSE во времени для всех сенсоров
ax2 = axes[0, 1]
sensor_names = [f'Сенсор {i+1}' for i in range(len(mse_history))]
ax2.plot(time_points, mse_history.T, alpha=0.7, linewidth=1)
ax2.plot(time_points, np.mean(mse_history, axis=0), 'k-', linewidth=3, 
         label='Средняя ошибка')
ax2.set_title('Ошибка MSE во времени для всех сенсоров', fontsize=12)
ax2.set_xlabel('Время (с)')
ax2.set_ylabel('MSE (мВ²)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')

# 3. Сравнение нескольких сенсоров в фиксированное время
ax3 = axes[1, 0]
time_idx = len(time_points) // 2  # Среднее время
selected_sensors = np.linspace(0, len(sensor_positions)-1, 12, dtype=int)
ax3.scatter(sensor_positions[selected_sensors, 0], sensor_positions[selected_sensors, 1], 
           c=true_potential_temporal[selected_sensors, time_idx], 
           s=100, cmap='RdBu_r', label='Истинный', alpha=0.8)
scatter = ax3.scatter(sensor_positions[selected_sensors, 0], sensor_positions[selected_sensors, 1], 
                     c=predicted_potential_temporal[selected_sensors, time_idx], 
                     s=50, cmap='RdBu_r', marker='x', label='Восстановленный')
ax3.set_title(f'Сравнение потенциала на сенсорах\nв момент времени t={time_points[time_idx]:.2f}с', fontsize=12)
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.legend()
ax3.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax3, label='Потенциал (мВ)')

# 4. Распределение ошибок по сенсорам
ax4 = axes[1, 1]
final_errors = mse_history[:, -1]  # Ошибки в последний момент времени
ax4.hist(final_errors, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
ax4.axvline(np.mean(final_errors), color='red', linestyle='--', linewidth=2, 
           label=f'Среднее: {np.mean(final_errors):.4f}')
ax4.axvline(np.median(final_errors), color='orange', linestyle='--', linewidth=2,
           label=f'Медиана: {np.median(final_errors):.4f}')
ax4.set_title('Распределение ошибок по сенсорам\n(в последний момент времени)', fontsize=12)
ax4.set_xlabel('MSE (мВ²)')
ax4.set_ylabel('Количество сенсоров')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('figures/eeg_temporal_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ График временной эволюции потенциала создан")

In [ ]:
# 2. Визуализация 2D полей потенциала
print("🗺️  Создание 2D визуализации полей потенциала...")

# Извлекаем данные о полях потенциала
grid_positions = results['field_data']['positions']
grid_x = results['field_data']['x_grid']
grid_y = results['field_data']['y_grid']
n_x, n_y = len(grid_x), len(grid_y)
true_field_temporal = results['field_data']['true_potential_temporal']
predicted_field_temporal = results['field_data']['predicted_potential_temporal']

print(f"Размер сетки: {n_x} x {n_y} = {n_x * n_y} точек")
print(f"Временных шагов: {len(time_points)}")

# Выбираем несколько временных срезов
time_indices = [0, len(time_points)//4, len(time_points)//2, 3*len(time_points)//4, -1]
time_labels = [f't = {time_points[i]:.2f}с' for i in time_indices]

fig, axes = plt.subplots(3, 5, figsize=(20, 12))

for j, (time_idx, time_label) in enumerate(zip(time_indices, time_labels)):
    # Истинное поле
    ax_true = axes[0, j]
    true_field_slice = true_field_temporal[:, :, time_idx].reshape(n_x, n_y)
    im_true = ax_true.imshow(true_field_slice.T, extent=[grid_x[0], grid_x[-1], grid_y[0], grid_y[-1]], 
                           origin='lower', cmap='RdBu_r', aspect='auto')
    ax_true.set_title(f'Истинное поле\n{time_label}', fontsize=10)
    ax_true.set_xlabel('x')
    if j == 0:
        ax_true.set_ylabel('y')
    plt.colorbar(im_true, ax=ax_true, shrink=0.8)
    
    # Восстановленное поле
    ax_pred = axes[1, j]
    pred_field_slice = predicted_field_temporal[:, :, time_idx].reshape(n_x, n_y)
    im_pred = ax_pred.imshow(pred_field_slice.T, extent=[grid_x[0], grid_x[-1], grid_y[0], grid_y[-1]], 
                            origin='lower', cmap='RdBu_r', aspect='auto')
    ax_pred.set_title(f'Восстановленное поле\n{time_label}', fontsize=10)
    ax_pred.set_xlabel('x')
    if j == 0:
        ax_pred.set_ylabel('y')
    plt.colorbar(im_pred, ax=ax_pred, shrink=0.8)
    
    # Ошибка
    ax_error = axes[2, j]
    error_field = np.abs(true_field_slice - pred_field_slice)
    im_error = ax_error.imshow(error_field.T, extent=[grid_x[0], grid_x[-1], grid_y[0], grid_y[-1]], 
                              origin='lower', cmap='Reds', aspect='auto')
    ax_error.set_title(f'Абсолютная ошибка\n{time_label}', fontsize=10)
    ax_error.set_xlabel('x')
    if j == 0:
        ax_error.set_ylabel('y')
    plt.colorbar(im_error, ax=ax_error, shrink=0.8)

plt.tight_layout()
plt.savefig('figures/eeg_2d_fields.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 2D визуализация полей потенциала создана")

In [ ]:
# 3. 3D визуализация потенциала
print("🎯 Создание 3D визуализации потенциала...")

# Создаем 3D графики для разных времен
fig = plt.figure(figsize=(18, 12))

# Используем более редкую сетку для 3D визуализации
skip = 3  # Пропускаем точки для улучшения производительности
X_3d = grid_x[::skip]
Y_3d = grid_y[::skip]
X_mesh, Y_mesh = np.meshgrid(X_3d, Y_3d)

time_indices_3d = [0, len(time_points)//3, 2*len(time_points)//3, -1]
time_labels_3d = [f't = {time_points[i]:.2f}с' for i in time_indices_3d]

for j, (time_idx, time_label) in enumerate(zip(time_indices_3d, time_labels_3d)):
    # Истинное 3D поле
    ax1 = fig.add_subplot(2, 4, j+1, projection='3d')
    true_field_3d = true_field_temporal[::skip, ::skip, time_idx].reshape(len(X_3d), len(Y_3d))
    surf1 = ax1.plot_surface(X_mesh, Y_mesh, true_field_3d, cmap='RdBu_r', alpha=0.8)
    ax1.set_title(f'Истинное поле\n{time_label}', fontsize=10)
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel('Потенциал (мВ)')
    
    # Восстановленное 3D поле
    ax2 = fig.add_subplot(2, 4, j+5, projection='3d')
    pred_field_3d = predicted_field_temporal[::skip, ::skip, time_idx].reshape(len(X_3d), len(Y_3d))
    surf2 = ax2.plot_surface(X_mesh, Y_mesh, pred_field_3d, cmap='RdBu_r', alpha=0.8)
    ax2.set_title(f'Восстановленное поле\n{time_label}', fontsize=10)
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.set_zlabel('Потенциал (мВ)')

plt.tight_layout()
plt.savefig('figures/eeg_3d_fields.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 3D визуализация потенциала создана")

In [ ]:
# 4. Статистический анализ результатов
print("📊 Создание статистического анализа результатов...")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Эволюция средней ошибки во времени
ax1 = axes[0, 0]
mean_mse = np.mean(mse_history, axis=0)
std_mse = np.std(mse_history, axis=0)
ax1.plot(time_points, mean_mse, 'b-', linewidth=2, label='Средняя MSE')
ax1.fill_between(time_points, mean_mse - std_mse, mean_mse + std_mse, alpha=0.3, color='blue')
ax1.set_title('Эволюция ошибки обучения', fontsize=12)
ax1.set_xlabel('Время (с)')
ax1.set_ylabel('MSE (мВ²)')
ax1.grid(True, alpha=0.3)
ax1.legend()

# 2. Корреляция между истинным и восстановленным потенциалом
ax2 = axes[0, 1]
correlations = []
for t_idx in range(len(time_points)):
    corr = np.corrcoef(true_potential_temporal[:, t_idx], 
                      predicted_potential_temporal[:, t_idx])[0, 1]
    correlations.append(corr)

ax2.plot(time_points, correlations, 'g-', linewidth=2)
ax2.set_title('Корреляция истинного и восстановленного потенциала', fontsize=12)
ax2.set_xlabel('Время (с)')
ax2.set_ylabel('Корреляция')
ax2.set_ylim([0, 1])
ax2.grid(True, alpha=0.3)

# 3. Распределение потенциала по области
ax3 = axes[0, 2]
final_time_idx = -1
true_final_flat = true_field_temporal[:, :, final_time_idx].flatten()
pred_final_flat = predicted_field_temporal[:, :, final_time_idx].flatten()

ax3.hist(true_final_flat, bins=50, alpha=0.5, label='Истинное поле', density=True)
ax3.hist(pred_final_flat, bins=50, alpha=0.5, label='Восстановленное поле', density=True)
ax3.set_title('Распределение потенциала в области\n(последний момент времени)', fontsize=12)
ax3.set_xlabel('Потенциал (мВ)')
ax3.set_ylabel('Плотность')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Карта ошибок по области
ax4 = axes[1, 0]
final_error_field = np.abs(true_field_temporal[:, :, final_time_idx] - 
                          predicted_field_temporal[:, :, final_time_idx])
im4 = ax4.imshow(final_error_field.T, extent=[grid_x[0], grid_x[-1], grid_y[0], grid_y[-1]], 
                origin='lower', cmap='Reds')
ax4.set_title('Пространственное распределение ошибки\n(последний момент времени)', fontsize=12)
ax4.set_xlabel('x')
ax4.set_ylabel('y')
plt.colorbar(im4, ax=ax4, label='Абсолютная ошибка (мВ)')

# 5. Тепловая карта корреляций по сенсорам
ax5 = axes[1, 1]
sensor_correlation_matrix = np.zeros((len(sensor_positions), len(time_points)))
for i, sensor_pos in enumerate(sensor_positions):
    for t_idx in range(len(time_points)):
        # Находим ближайшую точку на сетке к позиции сенсора
        distances = np.sqrt((grid_positions[:, 0] - sensor_pos[0])**2 + 
                          (grid_positions[:, 1] - sensor_pos[1])**2)
        closest_idx = np.argmin(distances)
        grid_coord = np.unravel_index(closest_idx, (n_x, n_y))
        
        # Вычисляем корреляцию во времени для этой позиции
        true_slice = true_field_temporal[grid_coord[0], grid_coord[1], :]
        pred_slice = predicted_field_temporal[grid_coord[0], grid_coord[1], :]
        corr = np.corrcoef(true_slice, pred_slice)[0, 1]
        sensor_correlation_matrix[i, t_idx] = corr if not np.isnan(corr) else 0

im5 = ax5.imshow(sensor_correlation_matrix.T, aspect='auto', cmap='viridis', 
                extent=[0, len(sensor_positions), 0, len(time_points)])
ax5.set_title('Корреляция по сенсорам во времени', fontsize=12)
ax5.set_xlabel('Индекс сенсора')
ax5.set_ylabel('Индекс времени')
plt.colorbar(im5, ax=ax5, label='Корреляция')

# 6. Сводная статистика
ax6 = axes[1, 2]
ax6.axis('off')

# Вычисляем итоговые метрики
final_mean_error = np.mean(final_errors)
final_max_error = np.max(final_errors)
final_correlation = np.mean(correlations)
mean_field_error = np.mean(final_error_field)

stats_text = f"""
СВОДНАЯ СТАТИСТИКА

Метрики качества восстановления:
• Средняя MSE на сенсорах: {final_mean_error:.6f}
• Максимальная MSE: {final_max_error:.6f}
• Средняя корреляция: {final_correlation:.4f}
• Средняя ошибка в области: {mean_field_error:.4f} мВ

Параметры эксперимента:
• Количество сенсоров: {n_sensors}
• Временных шагов: {len(time_points)}
• Точек на сетке: {n_x} × {n_y} = {n_x*n_y}

Интерпретация:
• {'Отличное' if final_correlation > 0.9 else 'Хорошее' if final_correlation > 0.7 else 'Удовлетворительное'} качество восстановления
• {'Низкая' if final_mean_error < 0.01 else 'Средняя' if final_mean_error < 0.1 else 'Высокая'} ошибка восстановления
"""

ax6.text(0.05, 0.95, stats_text, transform=ax6.transAxes, fontsize=10,
         verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))

plt.tight_layout()
plt.savefig('figures/eeg_statistical_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Статистический анализ результатов создан")

In [ ]:
# Сохранение результатов
print("💾 Сохранение результатов...")

# Создаем словарь с результатами для сохранения
results_summary = {
    "experiment_info": {
        "n_sensors": n_sensors,
        "n_time_steps": len(time_points),
        "grid_size": f"{n_x}x{n_y}",
        "domain": {
            "spatial": spatial_domain,
            "temporal": temporal_domain
        }
    },
    "performance_metrics": {
        "final_mean_mse": float(np.mean(final_errors)),
        "final_max_mse": float(np.max(final_errors)),
        "mean_correlation": float(np.mean(correlations)),
        "mean_field_error": float(np.mean(final_error_field))
    },
    "neural_network_config": nn_config,
    "optimization_config": opt_config,
    "loss_config": loss_config
}

# Сохраняем в JSON
with open('examples/eeg_inverse_results.json', 'w', encoding='utf-8') as f:
    json.dump(results_summary, f, indent=2, ensure_ascii=False)

print("✅ Результаты сохранены в examples/eeg_inverse_results.json")
print("✅ Графики сохранены в папке figures/")

print("\n" + "="*60)
print("🎉 ЭКСПЕРИМЕНТ ОБРАТНОЙ ЗАДАЧИ ЭЭГ ЗАВЕРШЕН!")
print("="*60)
print(f"📊 Средняя ошибка восстановления: {final_mean_error:.6f}")
print(f"📈 Средняя корреляция: {final_correlation:.4f}")
print(f"🎯 Качество: {'Отличное' if final_correlation > 0.9 else 'Хорошее' if final_correlation > 0.7 else 'Удовлетворительное'}")
print("="*60)